# [Create Agent with Function Call](https://learn.microsoft.com/en-us/python/api/overview/azure/ai-projects-readme?view=azure-python-preview#create-agent-with-function-call) using Tools and Tool Resources
You can enhance your Agents by defining **callback functions as function tools**. These can be provided to `create_agent` via either the `toolset` parameter or the **combination** of `tools and tool_resources`. Here are the distinctions:

1. **toolset**: When using the toolset parameter, you provide not only the function definitions and descriptions but also their implementations. The SDK will execute these functions within `create_and_run_process` or `streaming`. These functions will be invoked based on their definitions.
2. **tools and tool_resources**: When using the tools and tool_resources parameters, only the function definitions and descriptions are provided to `create_agent`, without the implementations. The `Run` or event handler of stream will raise a `requires_action` status based on the function definitions. **Your code must handle this status and call the appropriate functions**.<br/>

As a reference point, let's see how we were used to managed this with [OpenAI Assistants API](https://github.com/maurominella/openai/blob/main/assistantapi/Assistant%20APIs%2002%20-%20SDK%20with%20Function%20Calling.ipynb)

In [1]:
import os, time
from dotenv import load_dotenv # requires python-dotenv
# import logging
# logging.basicConfig(level=logging.INFO) # Configure logging 

load_dotenv("./../config/credentials_my.env")
model_name =  "gpt-4o-0513" # https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/tools/azure-ai-search?tabs=azurecli%2Cpython&pivots=overview-azure-ai-search
index_name =  "ms-surface-specs"

print(f'Project Connection String: <...{os.environ["PROJECT_CONNECTION_STRING"][-30:]}>')

Project Connection String: <...mai04-rg;mmai-hub04-prj01-fvye>


In [2]:
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import FunctionTool, ToolSet, ToolOutput # <<<<<<<<<<<<<<< SPECIFIC FOR FUNCTION CALLING
from azure.identity import DefaultAzureCredential

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.environ["PROJECT_CONNECTION_STRING"]
)

project_client.scope

{'subscription_id': 'eca2eddb-0f0c-4351-a634-52751499eeea',
 'resource_group_name': 'mmai04-rg',
 'project_name': 'mmai-hub04-prj01-fvye'}

In [3]:
# define some custom functions

from datetime import date

def save_file(joke:str) -> str:
    """
    Saves a text file using the textual content passed in the joke variable.
    Args:
        joke (str): The text content to be saved into the file. 
        
    Returns:
        Optional[str]: Returns 'success: file saved' on success, otherwise returns an error message.
    """
    file_name = "joke.txt"
    try:
        with open(file_name, "w") as f:
            f.write(joke)
        return "success: file saved"
    except Exception as e:
        return f"error: {str(e)}"


def get_flights(date_1:date, date_2:date) -> int:
    """ Returns the number of flights in a date interval  """
    import json
    from dateutil.parser import parse
    flights = {
        "flights": abs((parse(date_2) - parse(date_1)).days) 
    }
    return json.dumps(flights)


def my_cat_born_date() -> date:
    """ Returns my cat's born date """
    import datetime, random, json
    from dateutil.relativedelta import relativedelta
    
    # Calculate the date as ten years ago  
    ten_years_ago = datetime.date.today() - relativedelta(years=10) 
    
    cat_born_date = {
        "cat_born_date": ten_years_ago.strftime("%Y-%m-%d")
    }
    return json.dumps(cat_born_date)

In [4]:
from typing import Any, Callable, Set

user_functions: Set[Callable[..., Any]] = {
    save_file, 
    get_flights, 
    my_cat_born_date
}

user_functions

{<function __main__.get_flights(date_1: datetime.date, date_2: datetime.date) -> int>,
 <function __main__.my_cat_born_date() -> datetime.date>,
 <function __main__.save_file(joke: str) -> str>}

In [5]:
# Let's use the "user_function" set, to call its functions

for function in user_functions:    
    if function.__name__ == "my_cat_born_date":
        cat_birthday = function() 
        print(f"My cat's birth date is {cat_birthday}")
    elif function.__name__ == "save_file": 
        result = function("This is the content")
        print(f"save_file() returned <{result}>")
    elif function.__name__ == "get_flights": 
        result = function("2015-01-01", "2021-04-04")
        print(f"get_flights() returned <{result}>")

save_file() returned <success: file saved>
My cat's birth date is {"cat_born_date": "2015-01-01"}
get_flights() returned <{"flights": 2285}>


In [6]:
functions = FunctionTool(user_functions)
functions.definitions

[{'type': 'function', 'function': {'name': 'save_file', 'description': 'Saves a text file using the textual content passed in the joke variable.', 'parameters': {'type': 'object', 'properties': {'joke': {'type': 'string', 'description': 'No description'}}, 'required': ['joke']}}},
 {'type': 'function', 'function': {'name': 'my_cat_born_date', 'description': "Returns my cat's born date ", 'parameters': {'type': 'object', 'properties': {}, 'required': []}}},
 {'type': 'function', 'function': {'name': 'get_flights', 'description': 'Returns the number of flights in a date interval  ', 'parameters': {'type': 'object', 'properties': {'date_1': {'type': 'string', 'description': 'No description'}, 'date_2': {'type': 'string', 'description': 'No description'}}, 'required': ['date_1', 'date_2']}}}]

In [7]:
# Check if we already have any agents defined
project_client.agents.list_agents()

{'object': 'list', 'data': [{'id': 'asst_PHk0MbZKRdMj3a1kANKRfpna', 'object': 'assistant', 'created_at': 1735763240, 'name': 'my-assistant', 'description': None, 'model': 'gpt-4o-0513', 'instructions': 'You are a helpful assistant', 'tools': [{'type': 'function', 'function': {'name': 'save_file', 'description': 'Saves a text file using the textual content passed in the joke variable.', 'parameters': {'type': 'object', 'properties': {'joke': {'type': 'string', 'description': 'No description'}}, 'required': ['joke']}, 'strict': False}}, {'type': 'function', 'function': {'name': 'my_cat_born_date', 'description': "Returns my cat's born date ", 'parameters': {'type': 'object', 'properties': {}, 'required': []}, 'strict': False}}, {'type': 'function', 'function': {'name': 'get_flights', 'description': 'Returns the number of flights in a date interval  ', 'parameters': {'type': 'object', 'properties': {'date_1': {'type': 'string', 'description': 'No description'}, 'date_2': {'type': 'string'

In [8]:
# Create agent with AI search tool and process assistant run
agent = project_client.agents.create_agent(
    model=model_name,
    name="my-assistant",
    instructions="You are a helpful assistant",
    tools=functions.definitions
)

agent.items

<bound method _MyMutableMapping.items of {'id': 'asst_OCVaN5cb7pgtXwPY6KhRIzMn', 'object': 'assistant', 'created_at': 1735763283, 'name': 'my-assistant', 'description': None, 'model': 'gpt-4o-0513', 'instructions': 'You are a helpful assistant', 'tools': [{'type': 'function', 'function': {'name': 'save_file', 'description': 'Saves a text file using the textual content passed in the joke variable.', 'parameters': {'type': 'object', 'properties': {'joke': {'type': 'string', 'description': 'No description'}}, 'required': ['joke']}, 'strict': False}}, {'type': 'function', 'function': {'name': 'my_cat_born_date', 'description': "Returns my cat's born date ", 'parameters': {'type': 'object', 'properties': {}, 'required': []}, 'strict': False}}, {'type': 'function', 'function': {'name': 'get_flights', 'description': 'Returns the number of flights in a date interval  ', 'parameters': {'type': 'object', 'properties': {'date_1': {'type': 'string', 'description': 'No description'}, 'date_2': {'ty

In [9]:
# Check again if the agent created above is now present here
project_client.agents.list_agents()

{'object': 'list', 'data': [{'id': 'asst_OCVaN5cb7pgtXwPY6KhRIzMn', 'object': 'assistant', 'created_at': 1735763283, 'name': 'my-assistant', 'description': None, 'model': 'gpt-4o-0513', 'instructions': 'You are a helpful assistant', 'tools': [{'type': 'function', 'function': {'name': 'save_file', 'description': 'Saves a text file using the textual content passed in the joke variable.', 'parameters': {'type': 'object', 'properties': {'joke': {'type': 'string', 'description': 'No description'}}, 'required': ['joke']}, 'strict': False}}, {'type': 'function', 'function': {'name': 'my_cat_born_date', 'description': "Returns my cat's born date ", 'parameters': {'type': 'object', 'properties': {}, 'required': []}, 'strict': False}}, {'type': 'function', 'function': {'name': 'get_flights', 'description': 'Returns the number of flights in a date interval  ', 'parameters': {'type': 'object', 'properties': {'date_1': {'type': 'string', 'description': 'No description'}, 'date_2': {'type': 'string'

In [10]:
# Create thread for communication
thread = project_client.agents.create_thread()
print(f"Created thread: {thread}")

Created thread: {'id': 'thread_c3WZv3OzeosSq4mEXrrSDfG0', 'object': 'thread', 'created_at': 1735763284, 'metadata': {}, 'tool_resources': {}}


In [11]:
# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="Please write into a file the nr of flights between my cat born date and Easter 2021.",
)
print(f"Created message: {message}")

Created message: {'id': 'msg_G1gozxsIPeY0VpMSAzwmLppN', 'object': 'thread.message', 'created_at': 1735763284, 'assistant_id': None, 'thread_id': 'thread_c3WZv3OzeosSq4mEXrrSDfG0', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'Please write into a file the nr of flights between my cat born date and Easter 2021.', 'annotations': []}}], 'attachments': [], 'metadata': {}}


In [12]:
%%time

run = project_client.agents.create_run(thread_id=thread.id, assistant_id=agent.id)

run = project_client.agents.get_run(thread_id=thread.id, run_id=run.id)
print(f"Run: {run}")

Run: {'id': 'run_xIucT4s2Zyc60maPTzP3pcGK', 'object': 'thread.run', 'created_at': 1735763285, 'assistant_id': 'asst_OCVaN5cb7pgtXwPY6KhRIzMn', 'thread_id': 'thread_c3WZv3OzeosSq4mEXrrSDfG0', 'status': 'in_progress', 'started_at': 1735763285, 'expires_at': 1735763885, 'cancelled_at': None, 'failed_at': None, 'completed_at': None, 'required_action': None, 'last_error': None, 'model': 'gpt-4o-0513', 'instructions': 'You are a helpful assistant', 'tools': [{'type': 'function', 'function': {'name': 'save_file', 'description': 'Saves a text file using the textual content passed in the joke variable.', 'parameters': {'type': 'object', 'properties': {'joke': {'type': 'string', 'description': 'No description'}}, 'required': ['joke']}, 'strict': False}}, {'type': 'function', 'function': {'name': 'my_cat_born_date', 'description': "Returns my cat's born date ", 'parameters': {'type': 'object', 'properties': {}, 'required': []}, 'strict': False}}, {'type': 'function', 'function': {'name': 'get_fli

In [15]:
# just for checking: analyze the current status

run = project_client.agents.get_run(thread_id=thread.id, run_id=run.id)
print(f"Current run status: {run.status}")
print(f"\nRequired action(s): {run.required_action}")
print(f"\nWe need to run {len(run.required_action.submit_tool_outputs.tool_calls)} tool call(s): {run.required_action.submit_tool_outputs.tool_calls}")

i = 0
tool_outputs = []
for tool_call in run.required_action.submit_tool_outputs.tool_calls:
    i += 1
    output = functions.execute(tool_call)
    print(f"\n{i} - Executing tool_call {tool_call.function.name} ({tool_call.id}) >>> output: {output}")
    tool_outputs.append(
        ToolOutput(
            tool_call_id=tool_call.id,
            output=output
        )
    )
    
run = project_client.agents.submit_tool_outputs_to_run(
    thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs
)

while run.status in ["queued", "in_progress"]:
    time.sleep(1)
    run = project_client.agents.get_run(thread_id=thread.id, run_id=run.id)
    print(f"Current run status: {run.status}")

Current run status: RunStatus.REQUIRES_ACTION

Required action(s): {'type': 'submit_tool_outputs', 'submit_tool_outputs': {'tool_calls': [{'id': 'call_SxRt6Z5XHTJyg2TeptU5eODU', 'type': 'function', 'function': {'name': 'save_file', 'arguments': '{"joke":"The number of flights between 2015-01-01 and 2021-04-04 is 2285."}'}}]}}

We need to run 1 tool call(s): [{'id': 'call_SxRt6Z5XHTJyg2TeptU5eODU', 'type': 'function', 'function': {'name': 'save_file', 'arguments': '{"joke":"The number of flights between 2015-01-01 and 2021-04-04 is 2285."}'}}]

1 - Executing tool_call save_file (call_SxRt6Z5XHTJyg2TeptU5eODU) >>> output: success: file saved
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.COMPLETED


In [16]:
# Fetch and log all messages
messages = project_client.agents.list_messages(thread_id=thread.id)
for m in messages.text_messages:
    print(m.text)

{'value': 'The number of flights between January 1, 2015 and April 4, 2021 (Easter 2021) has been written into a file successfully.', 'annotations': []}
{'value': 'Please write into a file the nr of flights between my cat born date and Easter 2021.', 'annotations': []}


In [17]:
# Get the last message from the sender
last_msg = messages.get_last_text_message_by_sender("assistant")
if last_msg:
    print(f"Last Message: {last_msg.text.value}")

Last Message: The number of flights between January 1, 2015 and April 4, 2021 (Easter 2021) has been written into a file successfully.


In [18]:
run_steps = project_client.agents.list_run_steps(run_id=run.id, thread_id=thread.id)
run_steps_data = run_steps['data']
for rs in run_steps["data"]:
    print(rs, '\n')

{'id': 'step_I5g4lGPZS5qmUcBvrpKQbpoC', 'object': 'thread.run.step', 'created_at': 1735763334, 'run_id': 'run_xIucT4s2Zyc60maPTzP3pcGK', 'assistant_id': 'asst_OCVaN5cb7pgtXwPY6KhRIzMn', 'thread_id': 'thread_c3WZv3OzeosSq4mEXrrSDfG0', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1735763335, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'message_creation', 'message_creation': {'message_id': 'msg_QuGuShjILpiIuSiRp5mn62QJ'}}, 'usage': {'prompt_tokens': 486, 'completion_tokens': 37, 'total_tokens': 523}} 

{'id': 'step_hpgR9gd0BaxTsEXDOAMfiFvT', 'object': 'thread.run.step', 'created_at': 1735763309, 'run_id': 'run_xIucT4s2Zyc60maPTzP3pcGK', 'assistant_id': 'asst_OCVaN5cb7pgtXwPY6KhRIzMn', 'thread_id': 'thread_c3WZv3OzeosSq4mEXrrSDfG0', 'type': 'tool_calls', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1735763333, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type'

# START teardown

In [19]:
project_client.agents.list_agents()

{'object': 'list', 'data': [{'id': 'asst_OCVaN5cb7pgtXwPY6KhRIzMn', 'object': 'assistant', 'created_at': 1735763283, 'name': 'my-assistant', 'description': None, 'model': 'gpt-4o-0513', 'instructions': 'You are a helpful assistant', 'tools': [{'type': 'function', 'function': {'name': 'save_file', 'description': 'Saves a text file using the textual content passed in the joke variable.', 'parameters': {'type': 'object', 'properties': {'joke': {'type': 'string', 'description': 'No description'}}, 'required': ['joke']}, 'strict': False}}, {'type': 'function', 'function': {'name': 'my_cat_born_date', 'description': "Returns my cat's born date ", 'parameters': {'type': 'object', 'properties': {}, 'required': []}, 'strict': False}}, {'type': 'function', 'function': {'name': 'get_flights', 'description': 'Returns the number of flights in a date interval  ', 'parameters': {'type': 'object', 'properties': {'date_1': {'type': 'string', 'description': 'No description'}, 'date_2': {'type': 'string'

In [20]:
print(f"deleting trhead: {thread}...")
project_client.agents.delete_thread(thread.id)

deleting trhead: {'id': 'thread_c3WZv3OzeosSq4mEXrrSDfG0', 'object': 'thread', 'created_at': 1735763284, 'metadata': {}, 'tool_resources': {}}...


{'id': 'thread_c3WZv3OzeosSq4mEXrrSDfG0', 'object': 'thread.deleted', 'deleted': True}

In [21]:
# Delete all agents
for pca in project_client.agents.list_agents()['data']:
    print(f"Deleting agent {pca.name} ({pca.id})...")
    project_client.agents.delete_agent(pca.id)

Deleting agent my-assistant (asst_OCVaN5cb7pgtXwPY6KhRIzMn)...
Deleting agent my-assistant (asst_PHk0MbZKRdMj3a1kANKRfpna)...
Deleting agent my-assistant (asst_vWnDF5YLLakSaVkAnYGaJYZC)...


# HIC SUNT LEONES